<a href="https://colab.research.google.com/github/thesis17/Afaan-Oromoo-chatGPT/blob/main/LSTM%26Transformer_CNN_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
alejopaullier_daigt_external_dataset_path = kagglehub.dataset_download('alejopaullier/daigt-external-dataset')
narsil_daigt_misc_path = kagglehub.dataset_download('narsil/daigt-misc')
darraghdog_hello_claude_1000_essays_from_anthropic_path = kagglehub.dataset_download('darraghdog/hello-claude-1000-essays-from-anthropic')
thedrcat_daigt_proper_train_dataset_path = kagglehub.dataset_download('thedrcat/daigt-proper-train-dataset')
kingki19_llm_generated_essay_using_palm_from_google_gen_ai_path = kagglehub.dataset_download('kingki19/llm-generated-essay-using-palm-from-google-gen-ai')
carlmcbrideellis_llm_7_prompt_training_dataset_path = kagglehub.dataset_download('carlmcbrideellis/llm-7-prompt-training-dataset')
thedrcat_daigt_v2_train_dataset_path = kagglehub.dataset_download('thedrcat/daigt-v2-train-dataset')
jdragonxherrera_augmented_data_for_llm_detect_ai_generated_text_path = kagglehub.dataset_download('jdragonxherrera/augmented-data-for-llm-detect-ai-generated-text')
nbroad_daigt_data_llama_70b_and_falcon180b_path = kagglehub.dataset_download('nbroad/daigt-data-llama-70b-and-falcon180b')
carlmcbrideellis_llm_mistral_7b_instruct_texts_path = kagglehub.dataset_download('carlmcbrideellis/llm-mistral-7b-instruct-texts')
datafan07_daigt_gemini_pro_8_5k_essays_path = kagglehub.dataset_download('datafan07/daigt-gemini-pro-8-5k-essays')
dardodel_4k_mixtral87b_crafted_essays_for_detect_ai_comp_path = kagglehub.dataset_download('dardodel/4k-mixtral87b-crafted-essays-for-detect-ai-comp')

print('Data source import complete.')


In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import string
import tensorflow_text as tf_text

In [ ]:
#train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
extra_train = pd.read_csv("/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv")
#extra_train_new = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv")
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
#train_prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

In [ ]:
extra_train_1 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v7.csv").drop(['prompt_id','prompt_name'], axis=1)
extra_train_2 = pd.read_csv("/kaggle/input/hello-claude-1000-essays-from-anthropic/persuade15_claude_instant1.csv").drop(['prompt_id','essay_title'],axis=1).rename(columns = {'essay_text': 'text'})
extra_train_2['generated'] = 1
extra_train_3 = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_falcon_v3.csv").drop(['prompt_name','model'], axis=1)
extra_train_4 = pd.read_csv("/kaggle/input/llm-generated-essay-using-palm-from-google-gen-ai/LLM_generated_essay_PaLM.csv").drop(['id','prompt_id'], axis=1)

extra_train_5 = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv").rename(columns = {'label': 'generated'}).drop(['prompt_name','source', 'RDizzl3_seven'], axis=1)

extra_train_5 = extra_train_5[extra_train_5['generated'] == 0]
extra_train_5 = extra_train_5.sample(n=18149)

extra_train_6 = pd.read_csv("/kaggle/input/4k-mixtral87b-crafted-essays-for-detect-ai-comp/Mixtral8x7b_4k_essays_for_DetectAIGeneratedTextCompetition.csv").rename(columns = {'AI_Essay': 'text'}).drop(['model_Raw_output','student_id','prompt_id','index'], axis=1)
extra_train_6['generated'] = 1

In [ ]:
extra_train_new = pd.concat([extra_train_1, extra_train_2,
                             extra_train_3, extra_train_4,
                             extra_train_5, extra_train_6], axis=0)

In [ ]:
extra_train_student_generated = extra_train[['text']]
extra_train_student_generated['generated'] = 0

extra_train_ai_generated = extra_train[['source_text']].rename(columns = {'source_text':'text'})
extra_train_ai_generated['generated'] = 1

extra_train_f = pd.concat([extra_train_student_generated, extra_train_ai_generated, extra_train_new], axis=0)
#extra_train_f = pd.concat([extra_train_student_generated, extra_train_ai_generated], axis=0)
extra_train_f

In [ ]:
#train_final = pd.concat([extra_train_f, train.drop(columns=['prompt_id', 'id'], axis=0)], axis=0).reset_index(drop=True).drop_duplicates()
train_final = extra_train_f
train_final

In [ ]:
import matplotlib.pyplot as plt
sequence_lengths = train_final['text'].apply(lambda x: len(x.split()))

plt.hist(sequence_lengths, bins=30, alpha=0.75, color='blue')
plt.title('Distribution of Text Sequence Lengths')
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(train_final['generated'].value_counts())

In [ ]:
train_df, val_df = train_test_split(train_final, test_size=0.3, random_state=222)

batch_size = 32

raw_train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df['text'].values, train_df['generated'].values)
).batch(batch_size)

raw_val_ds = tf.data.Dataset.from_tensor_slices(
    (val_df['text'].values, val_df['generated'].values)
).batch(batch_size)

In [ ]:
max_features = 75000
embedding_dim = 64
sequence_length = 512

def tf_lower_and_split_punct(text):
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text


# Text vectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_features,
    ngrams = (3,5),
    output_mode="int",
    output_sequence_length=sequence_length,
    pad_to_max_tokens=True
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [ ]:
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Conv1D, MaxPooling1D, Dense, Dropout, Concatenate, Flatten
from tensorflow.keras import Model, Input


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
def cnn_branch(inputs, kernel_sizes, filters, drop_rate):
    conv_blocks = []
    for kernel_size in kernel_sizes:
        conv = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')(inputs)
        conv = MaxPooling1D(pool_size=2)(conv)
        conv = Dropout(drop_rate)(conv)
        conv_blocks.append(conv)
    return Concatenate()(conv_blocks)

In [ ]:
def build_model(sequence_length, max_features, embedding_dim, num_heads, ff_dim, drop_rate):
    # 輸入層
    inputs = Input(shape=(sequence_length,), dtype="int64")

    # 嵌入層
    x = Embedding(input_dim=max_features, output_dim=embedding_dim)(inputs)

    # 雙向 LSTM 層
    x = Bidirectional(LSTM(64, return_sequences=True))(x)

    # Transformer Block
    x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim, rate=drop_rate)(x)

    # 多分支 CNN 模型
    x = cnn_branch(x, kernel_sizes=[3, 5], filters=32, drop_rate=drop_rate)

    # 額外的卷積與降維
    x = Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(drop_rate)(x)

    # 全連接層
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(drop_rate)(x)

    # 輸出層
    outputs = Dense(1, activation='sigmoid')(x)

    # 建立模型
    return Model(inputs=inputs, outputs=outputs)

In [ ]:
model = build_model(sequence_length=512, max_features=75000, embedding_dim=128,
                    num_heads=2, ff_dim=128, drop_rate=0.5)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["binary_accuracy"])

epochs = 1
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
test_text = test['text'].values
vectorized_test_text = vectorize_layer(test_text)
predictions = model.predict(vectorized_test_text)
test['generated'] = predictions
print(test[['text', 'generated']])

In [ ]:
test[['id', 'generated']].set_index('id').to_csv("submission.csv")